In [ ]:
import sys
import os
from template import *
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from datasets import load_dataset
from tqdm import tqdm 
from transformers import AutoModelForCausalLM,AutoTokenizer
from peft import AutoPeftModelForCausalLM
import torch
import json
from transformers import GenerationConfig,AutoConfig
from answer import ActivationLLama
from datasets import load_dataset, Dataset
from typing import Optional, Union
from peft import PeftModel


model_path = "Meta-Llama-3-8B-Instruct"
template_index = 1
peft = "RED" 
is_train_return = False 
no_repeat_ngram_size = 5 
peft_path = "/mnt/userdata/liangsirui/MyProject/RED/Results/Llama3/9000_attn_o_2e-5/template1/delta_vector.pth" 
repetition_penalty = 1.1 
data_num = 500 

if "attn_o" in peft_path:
    op_position = "attn_o"
elif "ffn" in peft_path:
    op_position = "ffn"

path = "/mnt/usercache/huggingface/"
if not os.path.exists(path):
    path = "/mnt/publiccache/huggingface/"
model_path = path + model_path
lora_path = "/mnt/userdata/liangsirui/MyProject/RED/Results/Llama3/9000_lora/template1"

model = AutoModelForCausalLM.from_pretrained(model_path,torch_dtype=torch.bfloat16,device_map="auto")

# lora
# model = PeftModel.from_pretrained(model, lora_path)

# RED
model = ActivationLLama(model, op_position=op_position)
model.load_model(peft_path)

tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="left", torch_dtype=torch.bfloat16)
tokenizer.padding_side = "left"
tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id

In [ ]:
question = "It takes 20 minutes for the oil to heat up to 300 degrees.  It then takes 40% longer for the oil to heat up to the desired temperature of 400 degrees.  After warming the oil it takes 5 minutes less time to cook than it took to warm up the oil.  How much time passes from starting the oil to having cooked chicken?"

# lora_step = "Find the number of people who think horse #2 will win the big race.\n20% of 50 people = 0.2 x 50 = 10 people\n\n"
# red_step = "We need to find out how many people think horse #1 will win. We know that 20% of 100 people think horse #3 will win. So, we can multiply 0.2 by 100 and then divide by 4 to get our answer.\n0.2 * 100 = 20\n20 / 4 = 5\nSo, 5 people think horse #6 will win."

# question = f"{question} Please select the correct next step. A: {lora_step} B: {red_step} The correct step is: "
# question = f"{question} Please select the correct next step. A: {red_step} B: {lora_step} The correct step is: "

prompt = prompt_template[1] 
question = prompt%question

question = question + "Step1, we should calculate the "

indexed_tokens = tokenizer.encode(question)
tokens = [tokenizer.decode(x) for x in indexed_tokens]
tokens_tensor = torch.tensor([indexed_tokens]).to("cuda")

with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

predicted_top10 = torch.argsort(outputs[0][0][-1], descending=True)[:10]
predicted_text = [tokenizer.decode(x) for x in predicted_top10]
print(predicted_text)

In [ ]:
# prompt = prompt_template[1] 
# question = prompt%question 

generation_config = GenerationConfig(
    do_sample=False,
    no_repeat_ngram_size=5, 
    repetition_penalty=1.1,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
)

prompt_ids = tokenizer(question, return_tensors='pt').to(model.base_model.device).input_ids
output_good = model.generate(input_ids=prompt_ids, max_new_tokens=300, generation_config=generation_config)

completion_good = tokenizer.decode(output_good[0], skip_special_tokens=True)
print(completion_good)

In [ ]:
question="There are three trees in Eddy's backyard. The shortest tree has a height of 6 feet, and the second tree has a height of 5 feet less than the shortest tree. The height of the tallest tree is twice the height of the two trees combined. How tall is the tallest tree?"

prompt = prompt_template[1]
question = prompt%question + "To solve this problem, we need to first find the height of the second tree. Second Tree = Shortest Tree"

prompt_ids = tokenizer(question, return_tensors='pt').to(model.base_model.device).input_ids
output = model(prompt_ids)

predicted_top10 = torch.argsort(output[0][0][-1], descending=True)[:50]
predicted_text = [tokenizer.decode(x) for x in predicted_top10]
print(predicted_text)

a_token_id = tokenizer.encode('+', add_special_tokens=False)[0]
b_token_id = tokenizer.encode('-', add_special_tokens=False)[0]
# c_token_id = tokenizer.encode('C', add_special_tokens=False)[0]
# d_token_id = tokenizer.encode('D', add_special_tokens=False)[0]

logits = output.logits[0, -1, :]  # shape: (vocab_size,)
import torch.nn.functional as F

probs = F.softmax(logits, dim=0)
a_prob = probs[a_token_id].item()
b_prob = probs[b_token_id].item()
# c_prob = probs[c_token_id].item()
# d_prob = probs[d_token_id].item()

print(f"Token '+' 的概率值: {a_prob})")
print(f"Token '-' 的概率值: {b_prob})")
# print(f"Token 'C' 的概率值: {c_prob})")
# print(f"Token 'D' 的概率值: {d_prob})")

In [7]:
import json
def read_json_file(file_path):
    try:
        # 以只读模式打开文件，使用 UTF-8 编码
        with open(file_path, 'r', encoding='utf-8') as file:
            # 加载 JSON 数据
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"文件 {file_path} 未找到。")
    except json.JSONDecodeError:
        print(f"文件 {file_path} 不是有效的 JSON 格式。")


def save_list_to_json(data_list, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)  # 使用 indent 格式化 JSON
    print(f"数据已成功存储到 {file_path}")
    
# file_path = "/mnt/userdata/liangsirui/MyProject/pyreft/dataset/interference/interference.json"
# interference = read_json_file(file_path) 


In [4]:
data0 = "/mnt/userdata/liangsirui/MyProject/Prefix/Results/Prefix_Output/Llama3/9000_math10k_all_8_2e-4/epoch3/gsm8k_eval.json"
data1 = "/mnt/userdata/liangsirui/MyProject/Prefix/Results/Prefix_Output/Llama3/9000_math10k_all_8_2e-4/epoch3/gsm8k_eval_reft.json"

data0 = read_json_file(data0) 
data1 = read_json_file(data1) 

In [11]:
question_count0 = 0
question_count1 = 0
question_dict = []

for key in data0.keys():
    if "eval" not in data0[key].keys():
        break
    if data0[key]["eval"] == True and data1[key]["eval"] == False:
        question_count0 += 1
        current_question = {}
        current_question['Question'] = data0[key]['Question']
        current_question['Output'] = data0[key]['Output']
        current_question['Answer'] = data0[key]['Answer']
        current_question['data0'] = data0[key]['attn_o']
        current_question['data0'] = current_question['data0'].replace('!','')
        current_question['data1'] = data1[key]['attn_o'].replace('!','')
        question_dict.append(current_question)

question_dict = question_dict
save_list_to_json(question_dict, "/mnt/userdata/liangsirui/MyProject/Prefix/question_example.json")

数据已成功存储到 /mnt/userdata/liangsirui/MyProject/Prefix/question_example.json


In [ ]:
file_path = "/mnt/userdata/liangsirui/MyProject/Prefix/Base_Results/Llama3/prm800k/prm800k_1.json"
interference = read_json_file(file_path)

In [ ]:
remove_list = []
for key in interference.keys():
    if interference[key]['Answer'] == None:
        remove_list.append(key)

for key in remove_list:
    interference.pop(key)

save_list_to_json(interference, file_path)    

In [5]:
def eval_one_data(model_answer: str, answer: str):
    
    if answer in model_answer:
        return True
    else:
        
        try:
            int_answer = float(answer)

            if int_answer.is_integer():
                int_answer = int(int_answer)
                formatted_answer = f"{int_answer:,}" if abs(int_answer) >= 1000 else str(int_answer)
            
            if answer in model_answer or str(int_answer) in model_answer or formatted_answer in model_answer:
                return True
            else:
                return False
            
        except:
            return False

In [14]:
# eval base模型生成的答案

data = read_json_file('/mnt/userdata/liangsirui/MyProject/Prefix/Base_Results/Llama3/prm800k/prm800k_8.json')

from tqdm import *
for key in tqdm(data.keys()):
    if eval_one_data(data[key]['base'], data[key]['Answer']):
        data[key]['eval'] = True
    else:
        data[key]['eval'] = False

save_list_to_json(data, '/mnt/userdata/liangsirui/MyProject/Prefix/Base_Results/Llama3/prm800k/prm800k_8.json')


100%|██████████| 9991/9991 [00:00<00:00, 66535.87it/s]


数据已成功存储到 /mnt/userdata/liangsirui/MyProject/Prefix/Base_Results/Llama3/prm800k/prm800k_8.json


In [15]:
prm800k = read_json_file('/mnt/userdata/liangsirui/MyProject/Prefix/Base_Results/Llama3/prm800k/prm800k_8.json')

train_dict = []

from tqdm import *
for key in tqdm(prm800k.keys()):
    if prm800k[key]['eval'] == True:
        train_dict.append(prm800k[key])

print('count:', len(train_dict))
save_list_to_json(train_dict, '/mnt/userdata/liangsirui/MyProject/Prefix/dataset/prm800k/clean_train.json')

100%|██████████| 9991/9991 [00:00<00:00, 1691298.03it/s]


count: 6450
数据已成功存储到 /mnt/userdata/liangsirui/MyProject/Prefix/dataset/prm800k/clean_train.json
